# Aula 06

Estamos trabalhando com 1309 observações de 12 variáveis. Para tornar as coisas um pouco mais explícitas, já que alguns dos nomes das variáveis não são totalmente claros, aqui está o que temos que lidar:

| Nome da Variável | Descrição                               |
|------------------|-----------------------------------------|
| Survived         | Sobreviveu (1) ou morreu (0)            |
| Pclass           | Classe do passageiro (1 = Upper Class, 2 = Middle Class, 3 = Lower Class) |
| Name             | Nome do passageiro                      |
| Sex              | Sexo do passageiro                      |
| Age              | Idade do passageiro                     |
| SibSp            | Número de irmãos/cônjuges a bordo       |
| Parch            | Número de pais/filhos a bordo           |
| Ticket           | Número do bilhete                       |
| Fare             | Tarifa                                  |
| Cabin            | Cabine                                  |
| Embarked         | Porto de embarque C = Cherbourg, Q = Queenstown, S = Southampton|

In [ ]:
import pandas as pd
import requests
from io import StringIO

# Criação do dataFrame dos alunos
# ID do arquivo no Google Drive
file_id = '1S5Nl793vcL5ZPTGjzKaIEbwbLaDplvIP'

# URL modificada para forçar o download do arquivo
url = f"https://drive.google.com/uc?id={file_id}"

# Tentando obter o arquivo com requests
try:
    response = requests.get(url)
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    data = pd.read_csv(csv_raw)
except requests.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")

display(type(data))
data.head()

Os nomes das pessoas não nos ajudam, mas mamos retirar os títulos das pessoas que estão a bordo. Perceba que em todos os casos o título da pessoa termina com '.' .

### Valores Ausentes

Vamos checar se existem valores ausentes ou NaN no dataset

A quantidade de valores ausentes em Idade, Embarque e Tarifa é pequena em comparação com a amostra total, mas cerca de 80% das informações da Cabine estão ausentes. Os valores ausentes em Idade, Embarque e Tarifa podem ser preenchidos com medidas estatísticas descritivas, mas isso não funcionaria para Cabine.

Valores ausentes na variável Idade são comumente preenchidos utilizando a mediana. Contudo, aplicar a mediana de todo o conjunto de dados pode não ser a abordagem mais eficaz, pois não considera variações dentro dos grupos de dados. Para refinar essa estimativa, é recomendável utilizar a estratificação por grupos relevantes, como as classes de passageiros (Pclass). 

A estratificação consiste em dividir o conjunto de dados em subgrupos mais homogêneos antes de calcular medidas estatísticas, como a mediana, para preenchimento de dados ausentes. Isso é especialmente útil quando há uma correlação significativa entre a variável com dados ausentes e a variável usada para estratificar.

Susbistituir os valores NaN pela mediana dos registos semelhantes.

Olhando outra vez os os NaN...

O atributo "Embarked" só possui dois valores ausentes, vamos subistituir pela label que mais apareceu.

A característica Cabine é um pouco complicada e precisa de mais exploração. Uma grande parte dos dados da Cabine está ausente e a característica em si não pode ser completamente ignorada, porque algumas cabines podem ter taxas de sobrevivência mais altas. Descobriu-se que a primeira letra dos valores de Cabine representa os decks onde as cabines estão localizadas. Esses decks eram principalmente separados por uma classe de passageiros, mas alguns deles eram usados por múltiplas classes de passageiros.

![alt text](https://vignette.wikia.nocookie.net/titanic/images/f/f9/Titanic_side_plan.png/revision/latest?cb=20180322183733)

* No Convés do Barco havia **6** salas rotuladas como **T, U, W, X, Y, Z**, mas apenas a cabine **T** está presente no conjunto de dados.
* Os conveses **A**, **B** e **C** eram exclusivos para passageiros da 1ª classe.
* Os conveses **D** e **E** eram para todas as classes.
* Os conveses **F** e **G** eram para passageiros da 2ª e 3ª classe.
* Ao ir de **A** para **G**, a distância até a escada aumenta, o que pode ser um fator de sobrevivência.

In [ ]:
for num in [1, 2, 3]:
    if num == 1:
        data.loc[data['Pclass'] == 1, 'Cabin'] = data.loc[data['Pclass'] == 1, 'Cabin'].fillna('ABC')
    elif num == 2:
        data.loc[data['Pclass'] == 2, 'Cabin'] = data.loc[data['Pclass'] == 2, 'Cabin'].fillna('DE')
    elif num == 3:
        data.loc[data['Pclass'] == 3, 'Cabin'] = data.loc[data['Pclass'] == 3, 'Cabin'].fillna('FG')